<h1>CS152 Assignment 1: The 8-puzzle</h1>

In this assignment, you are required to code a solution to the 8-puzzle, and its natural generalizations,
applying the A∗ search algorithm with various admissible heuristics.

<h2> Assignment Overview </h2>

1. Design a python class named PuzzleNode with appropriate attributes and methods to capture the state of the 8-puzzle and its generalizations, as well as the elements needed to implement an A∗ search. Your class should also contain a parent attribute to point to a parent node, which can be used after search to reconstruct the optimal solution. It should also implement the method $str(__self__)$ , which should print out a grid showing the board state.

2. Define the skeleton of a function solvePuzzle that accepts three arguments and returns three values. It should be callable by using the syntax 

    $$steps, frontierSize, err = solvePuzzle(n, state, heuristic, prnt)$$

    <u>Input Arguments:</u>
    - n - the puzzle dimension (i.e. n x n board)
    - state - the starting (scrambled) state of the puzzle, provided as a list of lists, with the blank space represented by the number 0. For example, for n=3, we could have state = [[7 2 4],[5 0 6],[8 3 1]] as in the image shown previously.
    - heuristic - a handle to a heuristic function (more details in the next step)
    - prnt - a boolean value that indicates whether or not to print the solution

   <u> Output Arguments: </u>
    - steps - the number of steps to optimally reach the goal state from the initial state
    - frontierSize - the maximum (i.e. worst-case) size of the frontier during the search
    - err - an error code (details given in the next step)

   Your function must follow this exact template, since your implementation will be tested as needed using another python script.


3. Implement the A∗ search within the body of solvePuzzle.

    1. Your implementation should first test whether or not the state provided is of the correct size and format, and contains every number from $0$ to $n^{2} − 1$ precisely once. If any of these are not satisfied, then the function should return error code -1, and zeros for the remaining outputs steps and frontierSize. You may put the code for this test inside another function if you wish and call this function from within solvePuzzle.
    
    2. Next, define two heuristic functions based on the board state: the number of misplaced tiles and the Manhattan distance. The heuristic functions should take a list of lists for the board state as above and return a numerical value, using a function call of the form `val = h1(state)`. The heuristic functions will be passed into solvePuzzle as defined in step 2. Once you have defined your heuristic functions, create a list of function handles to point to them in the main workspace (i.e. not a local variable inside a function). Call this list heuristics. The idea is that calling heuristics[0](state) will return the number of misplaced tiles, and heuristics[1](state) will return the Manhattan distance. The automated testing script will call each heuristic in turn. In the extension activities to follow, you can append any custom heuristic functions onto the end of the list
    
    3. Define the remainder of the A∗ search, using an appropriate data structure for the frontier, making use of your PuzzleNode class. The implementation should check for repeated states and avoid expanding these (including simply undoing the previous move). It should also check if a better path to a previously-visited state has been found at any search step and modify the frontier, as well as the relevant attributes of the the node pointing to that step accordingly. You may use a similar method to those adopted in Lesson 3.1.
    
    4. After having completed the search, return the maximum frontier size and number of
steps as required, and an error code of zero. If prnt is true, the function should additionally print out the states of the board (as lists of lists) from the initial scrambled state all
the way to the goal, as well as the number of moves to reach the goal and the maximum
frontier size. Each should be printed on its own line.


4. Run your code from the following scrambled 3 × 3 boards:
    - [[5,7,6],[2,4,3],[8,1,0]]
    - [[7,0,8],[4,6,1],[5,3,2]]
    - [[2,3,7],[1,8,0],[6,5,4]]
    
   Compare the performance of the two heuristics in terms of the number of moves and the frontier size from each of these initial conditions. You may automate this with another function if you wish. Document the results using a markdown cell if using a python notebook, or as a separate section in the PDF, making it clear which initial state lead to which result. Your code will be tested from other initial states also (with a possibly larger $n$) to ensure its correct operation
   
   
<h3> Extensions </h3>

1. [Extension to (3a)] If we were to simply place the numbers 1 - 8 in the nine squares of the 8-puzzle, we may end up in a configuration that is impossible to solve. However, we can show that this configuration can be resolved to a state that differs from the true solved state by switching the places of any two adjacent tiles, excluding the blank square (see exercise 3.4 in Russell & Norvig). In addition to testing the format and sizes of initial state provided, test whether or not the puzzle can actually be solved from that state in the first place in the solvePuzzlefunction, returning error code -2 if it cannot be solved.

2. [Extension to (3b) and (4)] Design and implement another heuristic for the 8-puzzle that outperforms both of the heuristics tested previously. This could include other more sophisticated
heuristics that have been investigated in the literature. Justify why this heuristic dominates
both the number or misplaced tiles and the Manhattan distance, then implement your heuristic and compare its performance as in the basic requirements section. Extra kudos will be
given for anyone who implements a heuristic based on a pattern database.

3. [Extension to (3c)] Speed up the execution of the code by saving the heuristic values previously evaluated for a given state, to avoid the heuristic function being evaluated again for a
different puzzle instance. You can use a python decorator for achieving this - this is known
as memoization. What this means is that subsequent calls to solvePuzzle will be sped up by
storing knowledge of the heuristic function from previous search runs. You could also use
memoization for constructing a pattern database in extension question 2. Note that you are
not required to store this information between successive runs of the python kernel.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
#Import packages needed to implement A* search for 8 puzzle problem
import itertools
import numpy as np
from copy import deepcopy
import heapq
import json
import math
from collections import deque

#Now, define the class PuzzleNode:
class PuzzleNode:
    
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    def __init__(self, state=[], parent= None, path_cost= 0, heuristic_cost= 0):
        
        """
        Class constructor to initialize attributes needed for A* search and solving the n^2-1 puzzle 
        
        Inputs:
            - state (list of lists): Current state of the puzzle board (default: [])
            - parent: Pointer to parent node which was acted on to on to get current board state (default: None)
            - path_cost: Cost of getting to current node from the initial node (default: 0)
            - heuristic_cost: Estimated cost of going from current node to goal node as determined by a 
              specific heuristic function (default: 0)
        """
        
        self.state= state
        self.parent= parent
        self.path_cost= path_cost
        self.h_cost= heuristic_cost
        
        #Estimated cost of the cheapest solution through the node
        #This will be used for comparison to determine which node in the p_queue should be expanded
        self.f_cost= path_cost+heuristic_cost  
        
        self.pruned = False #Indicate if node can be ignored during search
     
    def __lt__(self,other):
        """
        Comparison function based on f_cost to be used by Priority Queue for ordering.
        A smaller f_cost has higher priority
        """
        return self.f_cost < other.f_cost
    
    def __str__(self):
        
        """
        Print out a grid showing the current board state
        """
        return ("\n".join(str(row).replace(',','').replace('[','|').replace(']','|') for row in self.state))
                       

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
def memoize(func):
    """
    Store the results of the decorated function for fast lookup and avoidance of recomputation
    """
    
    #Store the results in a dictionary that maps the argument (state) to results
    cache= {}
    
    #Define the wrapper function to return
    def wrapper(arg):
        
        #If the argument (in this case the state) has not been seen before
        if str(arg) not in cache:
            
            #Call func() and store the result
            cache[str(arg)]= func(arg)
        
        #Return the function value
        return cache[str(arg)]
    
    return wrapper

# Memoized Misplaced tiles heuristic
@memoize
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    
    Input:
        - state: the board state as a list of lists
    Output:
        - h: the number of misplaced tiles
    """
    
    #-Flatten state and pair up each tile in state with the corresponding goal tile in its position
    #-Compare each tile with its matched goal tile and record mismatches
    h = [goal_tile!= current_tile for (goal_tile,current_tile) in enumerate(itertools.chain(*state))]
    
    #Return the number of misplaced tiles (blank space (0) excluded)
    return sum(h)-1
    
# Memoized Manhattan distance heuristic
@memoize
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        - state: the board state as a list of lists
    Output:
        - h: the Manhattan distance from the solved configuration
    """
    
    #Get size of the board
    board_size= len(state)
    
    #Determine the coordinate of each tile in the goal state
    goal_coords= [(x, y) for x in range(board_size) for y in range(board_size)]
    
    #Assign the coordinate of a given tile to its tile value
    goal_coords_dict= {tile: coordinate for tile,coordinate in enumerate(goal_coords)}
    
    #Initialize manhattan distance to 0
    h= 0
    
    #Iterate through each tile in the current state
    for x in range(board_size):
        for y in range(board_size):
            tile = state[x][y]
            
            #If not the blank space
            if tile != 0:
                
                #Get the actual goal coordinate of the tile 
                goal_x, goal_y = goal_coords_dict[tile]
                
                #Calculate the vertical + horizontal distance from the tile to its goal position 
                h += abs(x - goal_x) + abs(y - goal_y)
    
    #Return the manhattan distance from the goal state
    return h

In [3]:
def create_pattern_db(n):
    
    """
    Create a pattern database for a given puzzle dimension n, storing all the subproblems in an external json file
    
    Input:
        -n: size of puzzle
        
    """
    
    #Calculate the number of possible puzzles from the goal states
    #Only half of the possible permutations of the numbers are reachable from the goal state
    num_states= math.factorial(n**2)/2
    
    #Generate goal state (in lists of lists form) for given dimensions
    goal= [[x for x in range(row*n,(row*n)+n)] for row in range(n)]
    
    #Defining dependencies and begginning Breadth First Search (BFS) to compute all possible solutions
    
    #Initialize a frontier with the goal state and the current cost of 0
    frontier= deque([[goal,0]])
    
    ##Initialize dictionaries to store explored states & solutions with the goal state
    solutions, explored= {str(goal):0}, set()
    
    print("Generating pattern database....")
    print("Storing cost of solutions.....")
    
    #While there are nodes in the frontier
    while frontier:
        
        #Choose current first node in the frontier
        node = frontier.popleft()
        
        #Get its state and the cost to get to the node from the goal state 
        state = node[0]
        cost = node[1]
        
        #Add 1 to the cost as all the successor nodes that will be generated below incur an additional
        #cost of 1 from the goal state
        successor_cost= cost+1
        
        #Get the different successor states after performing all valid moves of the blank space
        for successor in get_successors(state):
                
            #If the child node is already in the explored set, see if we need to update the path.
            if str(successor) in explored:
                if solutions[str(successor)] > successor_cost:
                    solutions[str(successor)] = successor_cost 
                else:
                    continue #Ignore this child, since a better path has been found previously.
            
            #If the child node has not been explored yet
            else:
                
                #Add to the frontier, record the cost to get to the child node, and mark it as explored
                frontier.append([successor,successor_cost])
                solutions[str(successor)]= successor_cost
                explored.add(str(successor))
                
        # Exit the loop when all permutations for the puzzle from the goal state have been found.
        if len(solutions) >= num_states:
            break
    
    print("Writing entries to external pattern database....")
    
    # Write solutions to a json file.
    with open('pattern_db.json', 'w') as fp:
        json.dump(solutions, fp)
    
    print("\n")
    print("Pattern database successfully generated.")
    

### Discussion of Heuristic 3

The third heuristic which implements a pattern database containing the exact number of moves (cost) to the goal state is guaranteed to outperform the two heuristics above in terms of node expansion for that exact reason. As the heuristic value for a node is exactly equal to the cost of moving from n to the goal, A* will behave perfectly (only follow the best path) and never expand anything else, making it very fast. This differs from the other heuristic values where A* could take wrong turns but will eventually still arrive at the optimal solution.

#### Admissibility and Consistency of Heuristic 3

It is clear that heuristic 3 is both admissible and consistent.

With regards to admissibility, we see that the cost to the goal is never overestimated (but is exact in this case) for any node by the heuristic. Hence, f(n) never overestimates the true cost of a solution along the current path through n.

As for consistency, since a uniform step cost was assigned in the precomputation of the solutions, we note that for every node $n$ and every successor $n'$ of $n$, the estimated cost of reaching the goal from $n$ is no greater than the step cost of getting to $n'$ plus the estimated cost of reaching the goal from $n'$.  

It is important to note that the time taken to compute all solutions is quite significant. However, this is amortized after time through speedy lookup and accuracy of solutions.

In [4]:
#Create a set to note which databases have been created for heuristic 3 usage
built_dbs= set()

def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state.
    It is implemented with a pattern database which contains the precomputed distances from the goal state
    to all possible states in this given puzzle. This distance will serve as the heuristic value
    
    
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the heuristic distance of the state from its solved configuration
        
    Note: This should eventually work for all puzzle dimensions but will be impractical with board size > 3
    """
    
    #Create and load a pattern database for this particular puzzle dimension if it has not been 
    #previously created
    if len(state) not in built_dbs:
        create_pattern_db(len(state))
        built_dbs.add(len(state))
        
        #Load the data into a dictionary called heuristic_vals and make it global to avoid recomputation
        with open('pattern_db.json') as f:
            global heuristic_vals
            heuristic_vals = json.load(f)
    
    #Return the cost from the goal state of the inputted state
    return heuristic_vals[str(state)]
    
#Memoized Linear conflict + Manhattan Distance
@memoize
def h4(state):
    """
    This function returns the sum of the manhattan distance and the least number of moves required to 
    resolve all linear conflicts in the given state.
    
    LINEAR CONFLICT DESCRIPTION:
    Two tiles ‘a’ and ‘b’ are in a linear conflict if they are in the same row or column,also their 
    goal positions are in the same row or column and the goal position of one of the tiles is blocked 
    by the other tile in that row.

    Input:
        -state: the board state as a list of lists
    Output:
        -h: the heuristic distance of the state from its solved configuration
    """
    
    #Get the dimension of the board
    board_size= len(state)
    
    #Generate goal state (in lists of lists form) for given dimensions
    goal= [[x for x in range(row*board_size,(row*board_size)+board_size)] for row in range(board_size)]
    
    
    #Initialize counter to record the number of linear conflicts
    sum_conflict = 0
    
    
    ### CALCULTATING HORIZONTAL LINEAR CONFLICT (NO. OF LINEAR CONFLICTS ON EACH ROW)
    
    #For each row
    for i in range(0, board_size):
        #For each tile
        for j in range(0, board_size):
            
            #If the not the blank tile
            if state[i][j] != '0':
                
                #Check all consecutive tiles on the row
                for k in range(j + 1, board_size):
                    
                    #If two tiles are on their goal row but are wrongly positioned, record linear conflict
                    if state[i][k] != '0' \
                            and state[i][j] in goal[i] \
                            and state[i][k] in goal[i] \
                            and goal[i].index(state[i][j]) > goal[i].index(state[i][k]):
                        sum_conflict += 1
    
    ### CALCULTATING VERTICAL LINEAR CONFLICT (NO. OF LINEAR CONFLICTS ON EACH COLUMN)
    #For each column in the puzzle
    for column in range(0, board_size):
        #For each tile in a given column
        for tile in range(0, board_size):
            
            #If not the blank tile
            if state[tile][column] != '0':
                
                #Check all consecutive tiles on the column
                for k in range(tile + 1, board_size):
                    
                    #Get current column
                    current_column= list(zip(*goal))[column]
                    
                    #If two tiles are on their goal column but are wrongly positioned, record linear conflict
                    if state[k][column] != '0' \
                            and state[tile][column] in current_column \
                            and state[k][column] in current_column \
                            and current_column.index(state[tile][column]) > current_column.index(
                        state[k][column]):
                        sum_conflict += 1
        
        
    return h2(state) + 2*sum_conflict
    
# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3, h4]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [5]:
def solvability_test(state):
    
    """
    Determine if a puzzle is solvable based on the number of inversions (total sum of the number of tiles with
    lower numbers preceded by a tile with a larger number for each tile)of the board and the board width
    
    RULES FOR DETERMINATION (Ryan, 2004):
    - If the grid width is odd, then the number of inversions in a solvable state is even.
    - If the grid width is even, and the blank is on an even row counting from the bottom, then the number of
      inversions in a solvable state is odd.
    - If the grid width is even, and the blank is on an odd row counting from the bottom 
      (last, third-last, fifth-last,etc) then the number of inversions in a solvable state is even.
      
    Input:
        - state (list of lists): the current state of the board
    
    Output:
        - True or False (boolean): Returns True if Puzzle is solvable and False otherwise
    """
    
    #Check if the state is in the right format
    if type(state) != list:
        raise TypeError('State must be a list of lists')

    #Initialize counter to record number of inversions. 
    inversions = 0
    
    #Get board with and the number of tiles
    board_width = len(state)
    num_tiles = board_width**2
    
    
    #If the Puzzle is not in the right dimension, reshape it
    state= np.array(state)
    if state.shape != (board_width,board_width):
        state = state.reshape(board_width, board_width) 
    
    #Find the row of the blank space. This is necessary for checking solvability as outlined above
    blank_tile_row = np.where(state==0)[0][0]
    
    #Flatten the state to a list to allow easy computation of inversions
    tiles = [*state.flatten()]  
    
    #Calculate the number of inversions
    for i in range(num_tiles - 1):
        for j in range(i+1, num_tiles):
            if (tiles[j] and tiles[i] != 0) and tiles[i] > tiles[j]:
                inversions += 1
                
    #Scenarios for solvability as outlined above
    
    #Board of odd width and even number of inversions
    if (board_width % 2 != 0) and (inversions % 2 == 0):
        return True
    
    #Board of even width, blank on an even row counting from the bottom, and odd number of inversions
    elif (board_width % 2 == 0) and (blank_tile_row % 2 != 0) and (inversions % 2 != 0):
        return True 
    
    #Board of even width, blank on an odd row counting from the bottom, and even number of inversions
    elif (board_width % 2 == 0) and (blank_tile_row % 2 == 0) and (inversions % 2 == 0):
        return True
    
    #If none of the cases above, the board is not solvable
    else:
        return False
        

In [6]:
def get_successors(state):
    
    """
    Generates successor states by moving blank space in possible directions given its current location.
    
    Input: 
        - state (list of lists): the current board state
    
    Output:
        - successors_list: the possible successor board states after applying all possible actions
    """
    
    # Find the current coordinates of the blank tile
    blank_position= np.where(np.array(state) == 0)
    row,col = blank_position[0][0], blank_position[1][0]
    
    #Create list to store successor states
    successors_list = []
     
    #Move blank space up if not on the first row
    if row != 0: 
        
        #Create a copy of the state to perform valid action on
        next_state = deepcopy(state)
        
        #Move the blank space up and record the successor state
        next_state[row][col], next_state[row-1][col] = next_state[row-1][col], next_state[row][col]
        successors_list.append(next_state)
    
    #Move blank space left if not in the first column
    if col != 0:
        
        #Create a copy of the state to perform valid action on
        next_state = deepcopy(state)
        
        #Move the blank space left and record the successor state
        next_state[row][col], next_state[row][col-1] = state[row][col-1], state[row][col]
        successors_list.append(next_state)
    
    #Move blank space down if not on the last row
    if row != len(state)-1: 
        
        #Create a copy of the state to perform valid action on
        next_state = deepcopy(state)
        
        #Move the blank space down and record the successor state
        next_state[row][col], next_state[row+1][col] = state[row+1][col], state[row][col]
        successors_list.append(next_state)

    #Move blank space right if not in the last column
    if col != len(state)-1: 
        
        #Create a copy of the state to perform valid action on
        next_state = deepcopy(state) 
        
        #Move the blank space right and record the successor state
        next_state[row][col], next_state[row][col+1] = state[row][col+1], state[row][col]
        successors_list.append(next_state)
    
    #Return the list of successor states
    return successors_list

def generate_goal(state):
    
    """
    Generate the goal state for a Puzzle given its dimension
    
    Input:
        - state (list of lists): the current state of the board
    """
    
    #Get the dimension of the board
    board_size= len(state)
    
    #Generate goal state (in lists of lists form) for given dimensions
    return [[x for x in range(row*board_size,(row*board_size)+board_size)] for row in range(board_size)]
    

In [7]:
def solvePuzzle(state, heuristic, show= False):
    
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    
    A* n-puzzle solver, adapted from A* Search in Python for Robot Navigation (Session 3.1) by R. Shekhar(2017) 
    
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
        
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of 
         lists that represents the state of the board at the ith step of the solution.
        -err: An error code. If state is not of the appropriate size and dimension, return -1. For the extention 
         task, if the state is not solvable, then return -2
    """
    
    #Initialize relevant counters or variables
    steps = 0
    exp= 0
    max_frontier = 0 
    err = 0     
    
    """
    EXTENSION 1 INCLUDED BELOW IN THE FORM OF 'solvability_test' FUNCTION DEFINED ABOVE
    """
    
    #If the inputted Puzzle is not solvable
    if solvability_test(state) is False:
        print("Error. The inputted puzzle is not solvable.")
        opt_path, err = None,-2
        return steps, exp, max_frontier, opt_path, err
    
    
    #Checking for correct dimensions
    board_size= len(state)
        
    #Check if the puzzle board is not a square (n*n) board
    for row in state:
        if len(row) != board_size:
            opt_path, err = None,-1
            return steps, exp, max_frontier, opt_path, err
    
    #Checking if the state contains the right numbers
    
    #The correct set of the numbers in sorted form (each number from 0 to n^2 − 1)
    correct_state = [*range(board_size**2)]
    
    #The sorted form of the numbers in the current state
    sorted_state= sorted([*itertools.chain(*state)])
    
    #Return False if the numbers in the sorted state is the not the correct set
    if sorted_state != correct_state:
        opt_path, err = None,-1
        return steps, exp, max_frontier, opt_path, err
    
    
    # Outlining dependencies and running the A* search algorithm:
    
    #Define the goal state for the given puzzle dimension
    goal_state = generate_goal(state)
    
    #Define the initial/root node
    initial_node = PuzzleNode(state,heuristic_cost=heuristic(state))
    
    #A dictionary to store visited PuzzleNodes. This prevents repeated states, including those which arise 
    #from simply undoing the previous move. 
    explored = {str(initial_node.state): initial_node}
    
    #Frontier, stored as a Priority Queue,
    #Add initial node to the priority queue and increment frontier size
    frontier = [initial_node]
    
    #While there are nodes in the frontier
    while frontier:
        
        #Get the node with the highest priority (lowest f_cost)
        current_node = heapq.heappop(frontier)
        
        #Check if node has been explored and marked for pruning
        if current_node.pruned:
            continue # Skip if it has
        
        # Check if the current_node is the goal state
        if current_node.state == goal_state: 
            break #End loop if it is
        
        #Get successor states (by moving blank space in all possible directions)
        successor_states = get_successors(current_node.state)
        
        #A node was expanded to yield successor states therefore increase the relevant counter
        exp +=1
        
        # Evaluating the next moves
        for successor in successor_states:
            #Tentative path cost to reach the node
            g_cost = current_node.path_cost + 1
            
            #If the child node is already in the explored set, see if we need to update the path.
            if str(successor) in explored:
                if explored[str(successor)].path_cost > g_cost:
                    explored[str(successor)].pruned = True # Mark existing value for deletion from frontier
                else:
                    continue # ignore this child, since a better path has been found previously.
                    
            # Create new node for successor
            child_node = PuzzleNode(successor, current_node, g_cost, heuristic(successor)) 
            
            #Add the successor/child node to the frontier and keep the highest priority node at the top
            #i.e. the node with the lowest f_cost
            heapq.heappush(frontier, child_node)
            
            #Update the maximum frontier size
            max_frontier= max(len(frontier), max_frontier)
            
            #Mark the child node as explored
            explored[str(successor)] = child_node 
    
    
    #Print out the progression from the initial to the goal state (optimal path)
    if show:
        
        # Compiling the optimal path to reach the goal state
        opt_path = [current_node]
        while current_node.parent: 
            opt_path.append((current_node.parent))
            current_node = current_node.parent
    
        #Reverse the paths as we want the path from the initial to the goal state
        opt_path.reverse()
        
        tag_str= ''
        
        for node in opt_path: 
            print(node,'\n')
            
        opt_path= [node.state for node in opt_path]
    
    #If not show
    else: 
        # Compiling the optimal path to reach the goal state
        opt_path = [current_node.state]
        while current_node.parent: 
            opt_path.append((current_node.parent).state)
            current_node = current_node.parent
    
        #Reverse the paths as we want the path from the initial to the goal state
        opt_path.reverse()
    
    return len(opt_path)-1, exp, max_frontier, opt_path, err
    

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [8]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [9]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [10]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [11]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [12]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

Error. The inputted puzzle is not solvable.


In [13]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

Generating pattern database....
Storing cost of solutions.....
Writing entries to external pattern database....


Pattern database successfully generated.


### Domination of Heuristic 1&2 By 3

From the above test, we see that the third heuristic (which is a precomputed exact solution) dominates the manhattan distance in all cases, and so in extension, it also dominates the misplaced tiles heuristic (which is dominated by the manhattan distance). More specifically, less nodes are expanded using heuristic 3 than heuristic 1 and 2. This is expected as heuristic three computes to exact cost to the goal state. Hence it leads A* on the perfect path with no needless node expanded. This differs from the manhattan and misplaces tiles heuristics which might traverse through unnessary nodes before eventually finding the optimal path. 

In [14]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[3])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

AssertionError: 

### Linear Conflict + Manhattan Distance vs. Manhattan Distances

From the above test, we see that the linear conflict heuristic does not dominate the manhattan distance as the number of nodes expanded was actually larger than that of the manhattan distance.

In [ ]:
## Memoization test - will be carried out after submission

<h1>References</h1>

Ryan, M. (2004, November 26). Solvability of the tiles game. School of Computer Science - University of Birmingham. https://www.cs.bham.ac.uk/~mdr/teaching/modules04/java2/TilesSolvability.html